In [1]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('gene_high_throughput_sequencing.csv', sep = ',')
data.head(10)

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917
5,STT5446_Breast_005_normal,normal,3.942275,4.488477,8.944837,12.581469,23.543887,8.683617,5.418139,50.525641,...,8.683617,1.314092,1.314092,1.314092,1.314092,1.314092,18.001936,1.314092,1.314092,1.314092
6,STT5451_Breast_006_normal,normal,1.084113,3.252340,8.917889,11.249037,19.179923,6.712171,12.439778,52.498406,...,9.091914,1.084113,1.084113,1.084113,1.084113,1.084113,11.659470,1.084113,1.084113,1.084113
7,STT5466_Breast_007_normal,normal,3.153900,1.647070,4.941211,11.529492,13.813151,8.235352,1.647070,44.226216,...,10.546396,1.647070,1.647070,1.647070,1.647070,1.647070,21.071346,1.647070,1.647070,1.647070
8,STT5472_Breast_008_normal,normal,2.551800,3.838876,16.255376,12.972108,20.169502,7.097877,5.333458,52.003891,...,9.274845,1.013982,1.013982,1.013982,1.013982,1.013982,11.030215,1.013982,1.013982,1.013982
9,STT5475_Breast_009_normal,normal,3.693128,1.231043,11.079385,12.453221,21.024058,6.780055,6.155214,49.174985,...,8.848678,1.231043,1.231043,1.231043,1.231043,1.231043,24.896300,1.231043,1.231043,1.231043


In [5]:
def write_answer(task,ans):
    with open("bio_" + str(task)+".txt", "w") as fout:
        fout.write(str(ans))

In [7]:
cancer_str = 'cancer'
early_neoplasia_str = 'early neoplasia'
normal_str = 'normal'

In [8]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *

In [12]:
p_values = pd.DataFrame(index = [0, 1], columns = data.columns[2:])
normal = data[data.Diagnosis == normal_str]
early = data[data.Diagnosis == early_neoplasia_str]
cancer = data[data.Diagnosis == cancer_str]

for column in p_values.columns:
    p_values[column][0] = scipy.stats.ttest_ind(normal[column], early[column], equal_var = False).pvalue
    p_values[column][1] = scipy.stats.ttest_ind(early[column], cancer[column], equal_var = False).pvalue


In [23]:
write_answer(1, (p_values.iloc[0] < 0.05).sum())
write_answer(2, (p_values.iloc[1] < 0.05).sum())

In [24]:
import statsmodels.stats.multitest as smm

In [53]:
def fold_change(C, T):
    if T > C:
        r = T/float(C)
    else:
        r = -C/float(T)
    return r

In [73]:
reject_0, p_corrected_0 = smm.multipletests(p_values.iloc[0], alpha=0.025, method='h')[:2]
result = 0
for i in range(len(reject_0)):
    if reject_0[i]:
        c = normal.iloc[:, i+2].mean()
        t = early.iloc[:, i+2].mean()
        if np.abs(fold_change(c,t)) > 1.5:
            result += 1
write_answer(3, result)

In [74]:
reject_1, p_corrected_0 = smm.multipletests(p_values.iloc[1], alpha=0.025, method='h')[:2]
result = 0
for i in range(len(reject_1)):
    if reject_1[i]:
        c = early.iloc[:, i+2].mean()
        t = cancer.iloc[:, i+2].mean()
        if np.abs(fold_change(c,t)) > 1.5:
            result += 1
write_answer(4, result)

In [75]:
reject_0_1, p_corrected_0 = smm.multipletests(p_values.iloc[0], alpha=0.025, method='fdr_bh')[:2]
result = 0
for i in range(len(reject_0_1)):
    if reject_0_1[i]:
        c = normal.iloc[:, i+2].mean()
        t = early.iloc[:, i+2].mean()
        if np.abs(fold_change(c,t)) > 1.5:
            result += 1
write_answer(5, result)

In [79]:
reject_1_1, p_corrected_0 = smm.multipletests(p_values.iloc[1], alpha=0.025, method='fdr_bh')[:2]
result = 0
for i in range(len(reject_1_1)):
    if reject_1_1[i]:
        c = early.iloc[:, i+2].mean()
        t = cancer.iloc[:, i+2].mean()
        if np.abs(fold_change(c,t)) >= 1.5:
            result += 1
write_answer(6, result)